<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250317%5D%20Thesis%20hybrid%20RF-LSTM%20with%20FGSM%20and%20PGD%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training on WSN-BFSF dataset (4 Classes)**

In [ ]:
#Imports various Python libraries necessary
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
#from keras.utils import np_utils
#from tensorflow.keras import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

In [ ]:
# Load dataset
dataset = pd.read_csv("dataset.csv")

# Preprocessing: Handle missing values (if any)
dataset = dataset.dropna() # Remove missing values

In [ ]:
dataset

,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num,Class
0,1,0.100000,79,79,600.000000,5,0,79.255,1.255,48,30,1,1,100,0,79,4,0
1,2,0.100963,78,78,599.979723,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
2,2,0.100963,76,76,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
3,2,0.100964,75,75,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
4,2,0.100964,118,118,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312101,4,656.526536,140,140,465.332078,5,800,140.255,1.255,48,30,1,31,100,16,140,68,1
312102,4,657.209643,140,140,465.191806,5,800,140.255,1.255,48,30,1,33,100,16,140,72,1
312103,4,657.891729,140,140,465.051737,5,800,140.255,1.255,48,30,1,35,100,16,140,76,1
312104,4,658.574656,140,140,464.911501,5,800,140.255,1.255,48,30,1,37,100,16,140,80,1


In [ ]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [262851  11766   7645  29844]


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler # Import StandardScaler here
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [262851 262851 262851 262851]


In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.regularizers import l2
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.regularizers import l2
import numpy as np

# Train RF model
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)

# Extract features from RF model
train_rf_features = rf_model.predict(X_train)
test_rf_features = rf_model.predict(X_test)

# Reshape RF features for LSTM input - Adding a new axis to create a 2D array before reshaping
train_rf_features = train_rf_features[:, np.newaxis]  # Reshape to (n_samples, 1)
test_rf_features = test_rf_features[:, np.newaxis]    # Reshape to (n_samples, 1)

# Reshape for LSTM input - Using the correct shape for 3D input
train_rf_features = np.reshape(train_rf_features, (train_rf_features.shape[0], 1, train_rf_features.shape[1]))
test_rf_features = np.reshape(test_rf_features, (test_rf_features.shape[0], 1, test_rf_features.shape[1]))

In [ ]:
# Train LSTM model with RF features
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(1, train_rf_features.shape[2]), return_sequences=True, kernel_regularizer=l2(0.0001)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.0001)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(16, kernel_regularizer=l2(0.0001)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(4, activation='softmax'))

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Convert y_train and y_test to one-hot encoded format
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=4)  # Assuming you have 4 classes
y_test = to_categorical(y_test, num_classes=4)
history = lstm_model.fit(train_rf_features, y_train, epochs=100, batch_size=64, validation_split=0.2)

# Evaluate hybrid model on test set
test_lstm_features = lstm_model.predict(test_rf_features)
test_rf_lstm_predictions = np.argmax(test_lstm_features, axis=1)

In [ ]:
# Plot training/validation loss over each epoch
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation/Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, classification_report
# Calculate evaluation metrics
accuracy = accuracy_score(np.argmax(y_test, axis=1), test_rf_lstm_predictions)
recall = recall_score(np.argmax(y_test, axis=1), test_rf_lstm_predictions, average='weighted')
precision = precision_score(np.argmax(y_test, axis=1), test_rf_lstm_predictions, average='weighted')
f1score = f1_score(np.argmax(y_test, axis=1), test_rf_lstm_predictions, average='weighted')
confusion_mat = confusion_matrix(np.argmax(y_test, axis=1), test_rf_lstm_predictions)
classification_rep = classification_report(np.argmax(y_test, axis=1), test_rf_lstm_predictions)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1-Score:", f1score)
print("Confusion Matrix:\n", confusion_mat)
print("Classification Report:\n", classification_rep)

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation/Test Loss')
plt.title('Training and Validation/Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# prompt: Display confusion matrix in heatmap

import seaborn as sns

# Assuming 'confusion_mat' is your confusion matrix from the previous code

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="bone",
            xticklabels=['Normal','Flooding','Blackhole','Forwarding'], # Replace with your actual class labels
            yticklabels=['Normal','Flooding','Blackhole','Forwarding']) # Replace with your actual class labels
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix Heatmap")
plt.show()


In [ ]:
# prompt: Display AUC and ROC curves with also show micro avg.

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Binarize the output
y_test_bin = label_binarize(np.argmax(y_test, axis=1), classes=[0, 1, 2, 3])
n_classes = y_test_bin.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], test_lstm_features[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), test_lstm_features.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curves for the multiclass problem
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.10f) for class 0' % roc_auc[0])
plt.plot(fpr[1], tpr[1], color='darkgreen',
         lw=lw, label='ROC curve (area = %0.10f) for class 1' % roc_auc[1])
plt.plot(fpr[2], tpr[2], color='darkblue',
         lw=lw, label='ROC curve (area = %0.10f) for class 2' % roc_auc[2])
plt.plot(fpr[3], tpr[3], color='darkred',
         lw=lw, label='ROC curve (area = %0.10f) for class 3' % roc_auc[3])

plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.10f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)


plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Example')
plt.legend(loc="lower right")
plt.show()

# Print AUC scores for each class and the micro average
print("AUC Score for class 0:", roc_auc[0])
print("AUC Score for class 1:", roc_auc[1])
print("AUC Score for class 2:", roc_auc[2])
print("AUC Score for class 3:", roc_auc[3])
print("Micro-average AUC Score:", roc_auc["micro"])


In [ ]:
# prompt: Display detection rates in each class

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Assuming 'confusion_mat' is your confusion matrix from the previous code
# and test_lstm_features and y_test are available

# ... (Your existing code)

# Binarize the output
y_test_bin = label_binarize(np.argmax(y_test, axis=1), classes=[0, 1, 2, 3])
n_classes = y_test_bin.shape[1]

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], test_lstm_features[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(f"Detection Rate for Class {i}: {roc_auc[i]}")


In [ ]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.9 MB/s eta 0:00:00


In [ ]:
# Train RF model for probability extraction
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Use RF probabilities instead of direct predictions
train_rf_probs = rf_model.predict_proba(X_train)
test_rf_probs = rf_model.predict_proba(X_test)

# Reshape for LSTM input
train_rf_features = np.reshape(train_rf_probs, (train_rf_probs.shape[0], 1, train_rf_probs.shape[1]))
test_rf_features = np.reshape(test_rf_probs, (test_rf_probs.shape[0], 1, test_rf_probs.shape[1]))

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
# One-hot encode target
y_train_enc = to_categorical(y_train)
y_test_enc = to_categorical(y_test)

# LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(train_rf_features.shape[1], train_rf_features.shape[2]), return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(16, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    Dense(y_train_enc.shape[1], activation='softmax')
])

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training LSTM
lstm_model.fit(train_rf_features, y_train_enc, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10515/10515 ━━━━━━━━━━━━━━━━━━━━ 68s 6ms/step - accuracy: 0.9870 - loss: 0.1088 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 2/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 78s 6ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 9.9669e-04
Epoch 3/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 61s 6ms/step - accuracy: 1.0000 - loss: 9.8498e-04 - val_accuracy: 1.0000 - val_loss: 8.0998e-04
Epoch 4/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - accuracy: 1.0000 - loss: 8.1698e-04 - val_accuracy: 1.0000 - val_loss: 6.3744e-04
Epoch 5/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 77s 5ms/step - accuracy: 1.0000 - loss: 6.9017e-04 - val_accuracy: 1.0000 - val_loss: 5.5232e-04
Epoch 6/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 62s 6ms/step - accuracy: 1.0000 - loss: 6.2496e-04 - val_accuracy: 1.0000 - val_loss: 5.6567e-04
Epoch 7/50
10515/10515 ━━━━━━━━━━━━━━━━━━━━ 76s 5ms/step - accuracy: 1.0000 - loss: 5.8268e-04 - val_accuracy: 1.0000 - val_loss: 4.8576e-04
Epoch 8/50
10515/10515 ━━━━━━━━━━━━━

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# ART Classifier setup
classifier = TensorFlowV2Classifier(
    model=lstm_model,
    nb_classes=y_train_enc.shape[1],
    input_shape=(train_rf_features.shape[1], train_rf_features.shape[2]),
    loss_object=CategoricalCrossentropy()
)

# FGSM attack generation
fgsm_attack = FastGradientMethod(estimator=classifier, eps=0.2)
X_test_adv_fgsm = fgsm_attack.generate(test_rf_features)

# Predictions
fgsm_predictions = np.argmax(classifier.predict(X_test_adv_fgsm), axis=1)

# Evaluation metrics
accuracy_fgsm = accuracy_score(y_test, fgsm_predictions)
precision_fgsm = precision_score(y_test, fgsm_predictions, average='weighted')
recall_fgsm = recall_score(y_test, fgsm_predictions, average='weighted')
f1_fgsm = f1_score(y_test, fgsm_predictions, average='weighted')

# Detailed classification report
report_fgsm = classification_report(y_test, fgsm_predictions)

# Print evaluation results
print(f"Accuracy under FGSM attack: {accuracy_fgsm:.10f}")
print(f"Precision under FGSM attack: {precision_fgsm:.10f}")
print(f"Recall under FGSM attack: {recall_fgsm:.10f}")
print(f"F1-Score under FGSM attack: {f1_fgsm:.10f}")

# Optional: print detailed classification report
print("\nDetailed Classification Report under FGSM Attack:\n")
print(report_fgsm)

Accuracy under FGSM attack: 0.9979693838
Precision under FGSM attack: 0.9979713140
Recall under FGSM attack: 0.9979693838
F1-Score under FGSM attack: 0.9979696790

Detailed Classification Report under FGSM Attack:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52570
           1       1.00      1.00      1.00     52570
           2       1.00      1.00      1.00     52570
           3       1.00      1.00      1.00     52571

    accuracy                           1.00    210281
   macro avg       1.00      1.00      1.00    210281
weighted avg       1.00      1.00      1.00    210281



In [ ]:
# PGD Attack with optimized parameters
pgd_attack = ProjectedGradientDescent(estimator=classifier, eps=0.2, max_iter=20, batch_size=32)
X_test_adv_pgd = pgd_attack.generate(test_rf_features)

# Predictions
pgd_predictions = np.argmax(classifier.predict(X_test_adv_pgd), axis=1)

# Evaluation metrics
accuracy_pgd = accuracy_score(y_test, pgd_predictions)
precision_pgd = precision_score(y_test, pgd_predictions, average='weighted')
recall_pgd = recall_score(y_test, pgd_predictions, average='weighted')
f1_pgd = f1_score(y_test, pgd_predictions, average='weighted')

# Detailed classification report
report_pgd = classification_report(y_test, pgd_predictions)

# Print evaluation results
print(f"Accuracy under PGD attack: {accuracy_pgd:.10f}")
print(f"Precision under PGD attack: {precision_pgd:.10f}")
print(f"Recall under PGD attack: {recall_pgd:.10f}")
print(f"F1-Score under PGD attack: {f1_pgd:.10f}")

# Optional: print detailed classification report
print("\nDetailed Classification Report under PGD Attack:\n")
print(report_pgd)

PGD - Batches: 0it [00:00, ?it/s]

Accuracy under PGD attack: 0.9979693838
Precision under PGD attack: 0.9979713140
Recall under PGD attack: 0.9979693838
F1-Score under PGD attack: 0.9979696790

Detailed Classification Report under PGD Attack:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52570
           1       1.00      1.00      1.00     52570
           2       1.00      1.00      1.00     52570
           3       1.00      1.00      1.00     52571

    accuracy                           1.00    210281
   macro avg       1.00      1.00      1.00    210281
weighted avg       1.00      1.00      1.00    210281

